In [8]:
import torch
import numpy as np
from PIL import Image, ImageStat
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import os
from torchvision import transforms 
import os
from openslide import OpenSlide
from openslide.deepzoom import DeepZoomGenerator
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import cv2


PIL.Image.MAX_IMAGE_PIXELS = None

: 

In [5]:
def get_level_for_magnification(slide, desired_mag, base_mag=40):
    """
    Map desired magnification to the appropriate pyramid level.
    """
    level = int(math.log(base_mag / desired_mag, 2))
    level = max(0, min(level, slide.level_count - 1))  # Clamp to valid levels
    return level

def is_tile_informative(tile, threshold=5.0):
    """
    Determine if a tile is informative based on the percentage of non-white pixels.

    Parameters:
    - tile (PIL.Image): The image tile.
    - threshold (float): Minimum percentage of non-white pixels to consider the tile informative.

    Returns:
    - bool: True if informative, False otherwise.
    """
    # Convert tile to NumPy array
    np_tile = np.array(tile)

    # If image has an alpha channel, ignore it
    if np_tile.shape[2] == 4:
        np_tile = np_tile[:, :, :3]

    # Calculate the percentage of non-white pixels
    # Assuming white background is (255, 255, 255)
    non_white = np.sum(np_tile < 250, axis=2)  # Pixels not near white
    total_pixels = np_tile.shape[0] * np_tile.shape[1]
    percent_non_white = (np.sum(non_white) / total_pixels) * 100

    return percent_non_white > threshold

def is_informative_tile(tile_image, std_threshold=10, mean_threshold=(30, 220), edge_threshold=1000):
    
    # Convert to grayscale for easier analysis
    grayscale_tile = tile_image.convert("L")
    stat = ImageStat.Stat(grayscale_tile)
    
    # Use the standard deviation as a measure of information content
    stddev = stat.stddev[0]
    mean = stat.mean[0]
    
    # Check if the tile is informative based on standard deviation and mean
    if not (stddev > std_threshold and mean_threshold[0] < mean < mean_threshold[1]):
        return False
    
    # Convert to numpy array for edge detection
    tile_array = np.array(grayscale_tile)
    edges = cv2.Canny(tile_array, 100, 200)
    edge_count = np.sum(edges > 0)
    
    # Check if the tile has enough edges to be considered informative
    return edge_count > edge_threshold

In [6]:
wsi_path = "C:/Users/User/Downloads/c7e229f4-7185-4211-979c-6cca1bbe4e0f/TCGA-AD-6890-01Z-00-DX1.4778042f-f210-489c-bb76-b4fe16b0d500.svs"  # Replace with your WSI file path
output_base_dir = "wsi"
tile_size = 512  # Size of the tile in pixels at the desired level
base_magnification = 40  # Adjust based on your WSI's base magnification
desired_magnifications = [10, 40]  # Add 100 if supported
overlap = 0  # Pixels to overlap between tiles
threshold = 5.0  # Minimum percentage of non-white pixels

    # Create base output directory
os.makedirs(output_base_dir, exist_ok=True)

    # Open the WSI file
slide = openslide.OpenSlide(wsi_path)

for mag in desired_magnifications:
        level = get_level_for_magnification(slide, mag, base_magnification)
        print(f"Processing tiles at {mag}x (level {level})")

        # Create output directory for this magnification
        output_dir = os.path.join(output_base_dir, f"{mag}x")
        os.makedirs(output_dir, exist_ok=True)

        # Extract and save informative tiles
        extract_informative_tiles(slide, level, tile_size, output_dir, overlap, threshold)

    # Close the slide
slide.close()
print("Tile extraction completed.")

Processing tiles at 10x (level 2)
Saved wsi\10x\tile_level2_x0_y0.png (Informative)
Saved wsi\10x\tile_level2_x0_y1.png (Informative)
Saved wsi\10x\tile_level2_x0_y2.png (Informative)
Saved wsi\10x\tile_level2_x0_y3.png (Informative)
Saved wsi\10x\tile_level2_x1_y0.png (Informative)
Saved wsi\10x\tile_level2_x1_y1.png (Informative)
Saved wsi\10x\tile_level2_x1_y2.png (Informative)
Saved wsi\10x\tile_level2_x1_y3.png (Informative)
Saved wsi\10x\tile_level2_x2_y0.png (Informative)
Saved wsi\10x\tile_level2_x2_y1.png (Informative)
Saved wsi\10x\tile_level2_x2_y2.png (Informative)
Saved wsi\10x\tile_level2_x2_y3.png (Informative)
Saved wsi\10x\tile_level2_x3_y0.png (Informative)
Saved wsi\10x\tile_level2_x3_y1.png (Informative)
Saved wsi\10x\tile_level2_x3_y2.png (Informative)
Saved wsi\10x\tile_level2_x3_y3.png (Informative)
Processing tiles at 40x (level 0)
Saved wsi\40x\tile_level0_x0_y0.png (Informative)
Saved wsi\40x\tile_level0_x0_y1.png (Informative)
Saved wsi\40x\tile_level0_x0_y2.